In [1]:
import h3
import osmnx as ox
import matplotlib.pyplot as plt

In [2]:
place_name = "Santa Rosa Island"

In [3]:
resolution = 10

In [4]:
graph_roads = ox.graph_from_place(place_name)

/root/anaconda3/envs/conda_py_3.9/lib/python3.9/site-packages/osmnx/geocoder.py:110: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(_geocode_query_to_gdf(q, wr, by_osmid))


In [5]:
nodes, edges_original = ox.graph_to_gdfs(graph_roads)

In [6]:
edges_original.head()

osmid        name highway  oneway    length  \
u         v          key                                                   
164742909 165088785  0    16219503   Main Road   track   False  1497.178   
          164743230  0    16219503   Main Road   track   False  2593.103   
          1133403183 0    16237006  Burma Road   track   False  2042.450   
164743230 164742909  0    16219503   Main Road   track   False  2593.103   
          164743307  0    16219503   Main Road   track   False   452.547   

                                                                   geometry  
u         v          key                                                     
164742909 165088785  0    LINESTRING (-120.11482 33.94541, -120.11479 33...  
          164743230  0    LINESTRING (-120.11482 33.94541, -120.11482 33...  
          1133403183 0    LINESTRING (-120.11482 33.94541, -120.11496 33...  
164743230 164742909  0    LINESTRING (-120.11706 33.92414, -120.11700 33...  
          164743307  0    LINESTRING (-120.11706 33.92414, -120.11716 33...

In [7]:
edges_original = edges_original.reset_index()
edges_original.head()

,u,v,key,osmid,name,highway,oneway,length,geometry
0,164742909,165088785,0,16219503,Main Road,track,False,1497.178,"LINESTRING (-120.11482 33.94541, -120.11479 33..."
1,164742909,164743230,0,16219503,Main Road,track,False,2593.103,"LINESTRING (-120.11482 33.94541, -120.11482 33..."
2,164742909,1133403183,0,16237006,Burma Road,track,False,2042.450,"LINESTRING (-120.11482 33.94541, -120.11496 33..."
3,164743230,164742909,0,16219503,Main Road,track,False,2593.103,"LINESTRING (-120.11706 33.92414, -120.11700 33..."
4,164743230,164743307,0,16219503,Main Road,track,False,452.547,"LINESTRING (-120.11706 33.92414, -120.11716 33..."


In [8]:
cols = ['geometry', 'name', 'highway']
edges_original = edges_original[cols]
edges_original.head()

,geometry,name,highway
0,"LINESTRING (-120.11482 33.94541, -120.11479 33...",Main Road,track
1,"LINESTRING (-120.11482 33.94541, -120.11482 33...",Main Road,track
2,"LINESTRING (-120.11482 33.94541, -120.11496 33...",Burma Road,track
3,"LINESTRING (-120.11706 33.92414, -120.11700 33...",Main Road,track
4,"LINESTRING (-120.11706 33.92414, -120.11716 33...",Main Road,track


In [9]:
edges_original.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [10]:
geometries = edges_original['geometry'].to_list()

In [11]:
len(geometries)

304

In [12]:
from shapely.geometry import Point
from shapely.geometry import LineString
import pyomnisci as po
import os
import pandas as pd
from shapely.geometry import Polygon
from shapely import wkt

In [13]:
def linestring_to_points(line):
    return line.coords

In [14]:
def get_geometry(h3_hex):
    points = h3.h3_to_geo_boundary(h3_hex, True)
    return Polygon(points)

In [18]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

In [24]:
def get_santa_rosa_roads_hexid_res(resolution,geometries):
    data = None
    final_df = pd.DataFrame(data, columns = ['name', 'highway','hex_id'])
    count = 0        # counter for every row of original dataframe
    deleted = 0

    for geometry in geometries:
        #print(f'''\ncalculating rows for geometry-{count}''')
        points = list(linestring_to_points(geometry))    
        hex_points = []    #hex points in linestring/geometry

        #looping through all points
        for i in range(1,len(points)):

            #h3 hex points corresponding to every point with resolution = 9
            #geo_to_h3 takes params in order lat,lng that is y,x of a point
            hex_point1 = h3.geo_to_h3(points[i-1][1],points[i-1][0],resolution)
            hex_point2 = h3.geo_to_h3(points[i][1],points[i][0],resolution)

            #getting list of h3 hexagons falling in the linestring 
            res = h3.h3_line(hex_point1,hex_point2)

            #adding h3 hexagons in list
            for j in range(0,len(res)):
                hex_points.append(res[j])

        #removing duplicate enteries from list
        hex_points_final = list(set(hex_points))

        #for every h3 hexagon we need to create a separate row, so looping through each entry and creating data for row 
        #and finally uploading to db

        df_arr = []
        #print(f'''creating dataframe for geometry-{count}, total rows: {len(hex_points_final)}''')
        for hex_point in hex_points_final:
            highway = ""
            if type(edges_original['highway'][count]) == list:
                highway = ' '.join(edges_original['highway'][count])
            else:
                highway = edges_original['highway'][count]

            name = ""
            if type(edges_original['name'][count]) == list:
                name = ' '.join(edges_original['name'][count])    
            else:
                name = edges_original['name'][count]

            row = {
                "name": [name],
                "highway":[highway],
                "hex_id":[hex_point]
                }
            df = pd.DataFrame(row)
            df_arr.append(df)

        #adding all the dataframe to one final dataframe
        tmp_df = None 
        if len(df_arr) > 0:
            tmp_df = pd.concat(df_arr)
            final_df = final_df.append(tmp_df)
        else:
            print(f'''skipping geometry-{count} as no hex found in this''')
        count +=1
    
    return final_df
    

In [40]:
santa_rosa_roads_hexid_res_10 = get_santa_rosa_roads_hexid_res(10,geometries)

In [41]:
santa_rosa_roads_hexid_res_10.reset_index(inplace = True)

In [43]:
santa_rosa_roads_hexid_res_10.head(2)

,index,name,highway,hex_id
0,0,Main Road,track,8a2912ac0487fff
1,0,Main Road,track,8a2912ac2bb7fff


In [44]:
santa_rosa_roads_hexid_res_10 = santa_rosa_roads_hexid_res_10[['name','highway','hex_id']]

In [46]:
output_path = "../Data/final_csv/santa_rosa_roads_hexid_res_10.csv"
santa_rosa_roads_hexid_res_10.to_csv(output_path)

In [47]:
santa_rosa_roads_hexid_res_10 = None

In [48]:
santa_rosa_roads_hexid_res_11 = get_santa_rosa_roads_hexid_res(11,geometries)
santa_rosa_roads_hexid_res_11.reset_index(inplace = True)
santa_rosa_roads_hexid_res_11 = santa_rosa_roads_hexid_res_11[['name','highway','hex_id']]
output_path = "../Data/final_csv/santa_rosa_roads_hexid_res_11.csv"
santa_rosa_roads_hexid_res_11.to_csv(output_path)
santa_rosa_roads_hexid_res_11 = None

In [49]:
santa_rosa_roads_hexid_res_12 = get_santa_rosa_roads_hexid_res(12,geometries)
santa_rosa_roads_hexid_res_12.reset_index(inplace = True)
santa_rosa_roads_hexid_res_12 = santa_rosa_roads_hexid_res_12[['name','highway','hex_id']]
output_path = "../Data/final_csv/santa_rosa_roads_hexid_res_12.csv"
santa_rosa_roads_hexid_res_12.to_csv(output_path)
santa_rosa_roads_hexid_res_12 = None